In [1]:
import os
import numpy as np
import pandas as pd
import pandas_gbq as pd_gbq
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from pandasql import sqldf
import datetime

In [2]:
pysqldf = lambda q: sqldf(q, globals())

df = pd.read_csv('GDT – BI Dev Challenge/challenge_A_task_02_dataset/challenge_A_task_02_star_schema_dataset/fct_listings.csv')

query = """
with fct_listings_cleaned as (
  select distinct
    df.listing_id
  , df.price
  , df.valid_from
  , case when df.valid_to is null and df2.valid_to is not null then df2.valid_to else df.valid_to end as valid_to
  , df.listing_date_key
  , df.platform_id
  , df.product_type_id
  , case when df.valid_to is null and df2.valid_to is not null then df2.status_id else df.status_id end as status_id
  , df.user_id
  from df
  left join df df2 on df.listing_id = df2.listing_id and df.listing_date_key = df2.listing_date_key and df.status_id != df2.status_id
)
select * from fct_listings_cleaned
"""

df = pysqldf(query)
df = df.fillna(value=np.nan)
df.to_csv('GDT – BI Dev Challenge/challenge_A_task_02_dataset/challenge_A_task_02_star_schema_dataset/fct_listings_clean.csv', index=False)
